In [19]:
import os
#os.chdir(r"C:\Users\Aditya\GitHub\zavmo\zavmo-api\zavmo")
os.chdir("/Users/adityachhabra/Github/zavmo/zavmo-api/zavmo")

In [20]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import Markdown

In [21]:
from helpers.agents.common import get_yaml_data,get_agent_instructions

In [22]:
from helpers.swarm import Agent, Response, Result, Tool, run_step
from helpers.agents.common import get_agent_instructions

In [18]:
active_agent   = discuss_agent
messages       = []
context        = {"email":"aditya@gmail.com","sequence_id":'52442',
                  "stage_data":{'profile':{},'discuss':{},'delivery':{},'demonstrate':{}},
                  "stage":1}

In [12]:
result = run_step(active_agent, messages, context=context, max_turns=5)

messages.extend(result.messages)
context.update(result.context)
if result.agent!=active_agent:
    active_agent = result.agent
    context['stage'] += 1
    print(f"Switching agent to {active_agent.name}")
print(context)

Markdown(messages[-1]['content'])

2024-10-24 23:53:41,628 - INFO - Running step 0 with agent Discussion
2024-10-24 23:53:42,841 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-24 23:53:42,846 - INFO - Updated context after update_discussion_data call: {'email': 'aditya@gmail.com', 'sequence_id': '52442', 'stage_data': {'discuss': {'curriculum': {'title': 'Web Development Bootcamp', 'subject': 'Web Development', 'level': 'Beginner', 'prerequisites': ['Basic computer literacy'], 'modules': [{'title': 'Introduction to HTML & CSS', 'learning_outcomes': ['Understand the structure of an HTML document', 'Style web pages using CSS'], 'lessons': ['HTML Basics', 'CSS Fundamentals', 'Building a Simple Webpage'], 'duration': 15}, {'title': 'JavaScript Essentials', 'learning_outcomes': ['Understand JavaScript syntax and control structures', 'Manipulate the DOM'], 'lessons': ['JavaScript Basics', 'Working with Functions', 'DOM Manipulation'], 'duration': 20}, {'title': 'Responsive Des

Transferring to Delivery Agent...


2024-10-24 23:53:44,167 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'email': 'aditya@gmail.com', 'sequence_id': '52442', 'stage_data': {'discuss': {'curriculum': {'title': 'Web Development Bootcamp', 'subject': 'Web Development', 'level': 'Beginner', 'prerequisites': ['Basic computer literacy'], 'modules': [{'title': 'Introduction to HTML & CSS', 'learning_outcomes': ['Understand the structure of an HTML document', 'Style web pages using CSS'], 'lessons': ['HTML Basics', 'CSS Fundamentals', 'Building a Simple Webpage'], 'duration': 15}, {'title': 'JavaScript Essentials', 'learning_outcomes': ['Understand JavaScript syntax and control structures', 'Manipulate the DOM'], 'lessons': ['JavaScript Basics', 'Working with Functions', 'DOM Manipulation'], 'duration': 20}, {'title': 'Responsive Design', 'learning_outcomes': ['Implement responsive layouts', 'Use media queries effectively'], 'lessons': ['Responsive Web Design Principles', 'CSS Flexbox', 'CSS Grid'], 'duration': 15}, {'title': 'Version Control with Git', 'learning_outcomes': ['Initialize and mana

Fantastic! 🎉 We've updated your learning preferences, and our next stage, the Delivery stage, is ready to kick off. This stage will provide you with the learning materials and resources tailored to your interests in Python programming with hands-on activities.

If you have any questions or need further assistance, feel free to reach out. Enjoy your learning journey! 🚀📚

In [11]:
user_resp = "Sounds good, let's go"

messages.append({"role":"user","content":user_resp})